# Merging genotypes

This script will merge several genotypes files
- Merging between participants' genotype platforms
- Merging reference samples with participants' samples

For reference samples we will use the 1000G and HGDP samples. 
To see the steps to merge the reference samples see [here](https://tomszar.github.io/HGDP_1000G_Merge/)

## Preliminaries

First let's import modules and set up paths

In [1]:
import glob, os, shutil, subprocess, csv, time
import pandas as pd
import numpy as np

In [2]:
projpath  = os.path.realpath('..')
pathgenos = os.path.join(projpath, "DataBases", "Genotypes")

## Merging in house genotypes

The first step will be to merge our participants' genotypes. 
The steps will be as follows:
- Clean each dataset by removing SNPs with missing rates greater than 0.1, SNPs with minor allele frequencies below 0.01, and a Hardy-Weinberg equilibrium exact test p-value below 1e-50
- Merge the datasets, flip strands, and remove possibly triallelic SNPs
- Finally, LD prune SNPs in the merged dataset

First we will do a cleaning in each data set to remove SNP with missing call rates greater than 0.1

In [3]:
#Clean datasets
os.chdir(pathgenos)
for file in glob.glob("01_Originals/*.bed"):
    inputname = file.split(".")
    outname = "Clean_" + inputname[0][13:]
    print("Creating..." + outname)
    subprocess.run(["plink", "--bfile", inputname[0], "--geno", "0.1", "--maf", "0.01", "--hwe", "1e-50", "--make-bed", "--out", "02_Cleaning/" + outname])
    
print("Finished")

Creating...Clean_UIUC2013_116ppl_959Ksnps_hg19_ATGC
Creating...Clean_Euro180_176ppl_317K_hg19_ATGC
Creating...Clean_CHP_1022ppl_114K_hg19_ATGC
Creating...Clean_SA_231ppl_599K_hg19_ATGC
Creating...Clean_TD_198ppl_1M_hg19_ATGC
Creating...Clean_ADAPT_2784ppl_1Msnps_hg19_ATGC
Creating...Clean_GHPAFF_3ppl_907K_hg19_ATGC
Creating...Clean_CV_697ppl_964K_hg19_ATGC
Creating...Clean_UIUC2014_168ppl_703K_hg19_ATGC
Finished


Here you can take a look at the loaded and removed SNPs in each dataset

In [4]:
for file in glob.glob("02_Cleaning/*.log"):
    with open(file) as myfile:
        print("In file: " + file.split(".")[0][12:])
        for num, line in enumerate(myfile, 1):
            if "variants" in line:
                print(line, end='')
        print("Finished file... \n")

In file: Clean_CHP_1022ppl_114K_hg19_ATGC
114495 variants loaded from .bim file.
1132 variants removed due to missing genotype data (--geno).
--hwe: 0 variants removed due to Hardy-Weinberg exact test.
753 variants removed due to minor allele threshold(s)
112610 variants and 1022 people pass filters and QC.
Finished file... 

In file: Clean_UIUC2013_116ppl_959Ksnps_hg19_ATGC
959382 variants loaded from .bim file.
30151 variants removed due to missing genotype data (--geno).
chromosome variants.
--hwe: 0 variants removed due to Hardy-Weinberg exact test.
52615 variants removed due to minor allele threshold(s)
876616 variants and 116 people pass filters and QC.
Finished file... 

In file: Clean_GHPAFF_3ppl_907K_hg19_ATGC
907494 variants loaded from .bim file.
52823 variants removed due to missing genotype data (--geno).
chromosome variants.
--hwe: 0 variants removed due to Hardy-Weinberg exact test.
376349 variants removed due to minor allele threshold(s)
478322 variants and 3 people pas

Now we will generate a first merging to get a list of problematic snps.
Based on comparing a few snps across datasets, it seems that the CV dataset contains most fliped snps, followed by the Euro dataset.
Then, we will flip the snps of the CV dataset, merge, flip the Euro dataset, and merge again.
Finally, we'll then extract the possibly triallelic snps from each dataset

In [ ]:
#First merge
subprocess.run(["plink", "--merge-list", "FirstMergeList.txt", "--out", "03_Merging/TriallelicSnps"])
#Flip the CV dataset
subprocess.run(["plink", "--bfile", "02_Cleaning/Clean_CV_697ppl_964K_hg19_ATGC", "--flip", "03_Merging/TriallelicSnps.missnp",
                "--make-bed", "--out", "02_Cleaning/Clean_CV_697ppl_964K_hg19_ATGC_flip"])
#Second merge
subprocess.run(["plink", "--merge-list", "SecondMergeList.txt", "--out", "03_Merging/TriallelicSnps_2"])
#Flip the Euro dataset
subprocess.run(["plink", "--bfile", "02_Cleaning/Clean_Euro180_176ppl_317K_hg19_ATGC", "--flip", "03_Merging/TriallelicSnps_2.missnp",
                "--make-bed", "--out", "02_Cleaning/Clean_Euro180_176ppl_317K_hg19_ATGC_flip"])
#Third merge
subprocess.run(["plink", "--merge-list", "ThirdMergeList.txt", "--out", "03_Merging/TriallelicSnps_3"])

#Get number of snps missing from first to third merge
num_lines  = sum(1 for line in open("03_Merging/TriallelicSnps.missnp"))
num_lines2 = sum(1 for line in open("03_Merging/TriallelicSnps_2.missnp"))
num_lines3 = sum(1 for line in open("03_Merging/TriallelicSnps_3.missnp"))
print(num_lines, num_lines2, num_lines3)

#removing snps from TriallelicSnps_3.missnp files from all datasets
for file in glob.glob("02_Cleaning/*.bed"):
    inputname = file.split(".")
    outname = "CleanTriallelic_" + inputname[0][18:]
    print("Creating..." + outname)
    subprocess.run(["plink", "--bfile", inputname[0], "--exclude", "03_Merging/TriallelicSnps_3.missnp", "--make-bed", "--out", "02_Cleaning/" + outname])
    
print("Finished")
print("Final merging")
subprocess.run(["plink", "--merge-list", "FinalMergeList.txt", "--out", "03_Merging/Merged"])

85589 16206 3786
Creating...CleanTriallelic_TD_198ppl_1M_hg19_ATGC
Creating...CleanTriallelic_GHPAFF_3ppl_907K_hg19_ATGC
Creating...CleanTriallelic_Euro180_176ppl_317K_hg19_ATGC
Creating...CleanTriallelic_UIUC2014_168ppl_703K_hg19_ATGC
Creating...CleanTriallelic_CV_697ppl_964K_hg19_ATGC
Creating...CleanTriallelic_Euro180_176ppl_317K_hg19_ATGC_flip
Creating...CleanTriallelic_SA_231ppl_599K_hg19_ATGC
Creating...CleanTriallelic_ADAPT_2784ppl_1Msnps_hg19_ATGC
Creating...CleanTriallelic_CHP_1022ppl_114K_hg19_ATGC
Creating...CleanTriallelic_CV_697ppl_964K_hg19_ATGC_flip
Creating...CleanTriallelic_UIUC2013_116ppl_959Ksnps_hg19_ATGC
Finished
Final merging


CompletedProcess(args=['plink', '--merge-list', 'FinalMergeList.txt', '--out', '03_Merging/Merged'], returncode=0)

Finally, we will remove all snps with missing call rates greater than 0.1, SNPs with minor allele frequencies below 0.01, and a Hardy-Weinberg equilibrium exact test p-value below 1e-50.
Finally, we will LD prune the set of SNPs using parameters 50, 5, and 2

In [ ]:
subprocess.run(["plink", "--bfile", "03_Merging/Merged", "--geno", "0.1", "--maf", "0.01", "--hwe", "1e-50", "--make-bed", "--out", "04_CleanMerged/Cleaned"])
subprocess.run(["plink", "--bfile", "04_CleanMerged/Cleaned", "--indep", "50", "5", "2", "--out", "04_CleanMerged/ExtractSNPs"])
with open("04_CleanMerged/ExtractSNPs.log") as myfile:
    for num, line in enumerate(myfile, 1):
        if "variants" in line:
            print(line, end='')
    print("Finished file... \n")

In [9]:
subprocess.run(["plink", "--bfile", "04_CleanMerged/Cleaned", "--extract", "04_CleanMerged/ExtractSNPs.prune.in", "--remove", "04_CleanMerged/ExtractSNPs.nosex", 
                "--make-bed", "--out", "04_CleanMerged/LDCleanMerged"])
with open("04_CleanMerged/CleanMerged.log") as myfile:
    for num, line in enumerate(myfile, 1):
        if "variants" in line:
            print(line, end='')
    print("Finished file... \n")

1365576 variants loaded from .bim file.
--extract: 19498 variants remaining.
19498 variants and 5290 people pass filters and QC.
Finished file... 



We'll remove intermediary datasets to clear space, and move the final dataset to be latter merged with the reference samples

In [10]:
#Remove source files
for f in glob.glob("02_Cleaning/*.*"):
    os.remove(f)
    
for f in glob.glob("03_Merging/*.*"):
    os.remove(f)

In [15]:
#Copy final file to Merge1000G to be merged with the 1000Genomes samples
dest_dir = os.path.join(projpath, 'DataBases', 'Genotypes', '05_ReferenceSamples')
for filename in glob.glob("04_CleanMerged/CleanMerged.*"):
    shutil.copy(filename, dest_dir)

In the next section delete the merging of the reference samples, and skip to the merging of the reference with our samples

## Merging reference and in-house samples

Finally, we will merge the in-house samples with the reference samples from HGDP and 1000G.
To do that we will extract the snps from the in-house samples already pruned and cleaned.
Follow these [steps](https://tomszar.github.io/HGDP_1000G_Merge/) to merge the reference samples. 
Paste the final file `hgdp1000ghg19` in the 05_ReferenceSamples folder

In [14]:
os.chdir(os.path.join(outhouse, "05_ReferenceSamples"))
subprocess.run(["plink", "--bfile", "hgdp1000ghg19", "--extract", "CleanMerged.bim", "--make-bed", "--out", "hgdp1000ghg19_subset"])
subprocess.run(["plink", "--bfile", "CleanMerged", "--extract", "hgdp1000ghg19_subset.bim", "--make-bed", "--out", "CleanMerged_subset"])
subprocess.run(["plink", "--bfile", "CleanMerged_subset", "--bmerge", "hgdp1000ghg19_subset", "--make-bed", "--out", "HouseHGDP1000Ghg19"])
#Fliping strand and merging
subprocess.run(["plink", "--bfile", "CleanMerged_subset", "--flip", "HouseHGDP1000Ghg19-merge.missnp", "--make-bed", "--out", "CleanMerged_subset_flip"])
subprocess.run(["plink", "--bfile", "CleanMerged_subset_flip", "--bmerge", "hgdp1000ghg19_subset", "--make-bed", "--out", "HouseHGDP1000Ghg19"])

with open("HouseHGDP1000Ghg19.log", 'r') as fin:
    file_contents = fin.read()
    print(file_contents)

PLINK v1.90b4 64-bit (20 Mar 2017)
Options in effect:
  --bfile CleanMerged_subset_flip
  --bmerge hgdp1000ghg19_subset
  --make-bed
  --out HouseHGDP1000Ghg19

Hostname: tomasgazelle
Working directory: /home/tomas/Downloads/FacialSD/Results/MergeGeno/MergeSamples/05_ReferenceSamples
Start time: Mon May 14 11:57:17 2018

Random number seed: 1526313437
3865 MB RAM detected; reserving 1932 MB for main workspace.
5290 people loaded from CleanMerged_subset_flip.fam.
3444 people to be merged from hgdp1000ghg19_subset.fam.
Of these, 3444 are new, while 0 are present in the base dataset.
12890 markers loaded from CleanMerged_subset_flip.bim.
12890 markers to be merged from hgdp1000ghg19_subset.bim.
Of these, 0 are new, while 12890 are present in the base dataset.
Performing single-pass merge (8734 people, 12890 variants).
Merged fileset written to HouseHGDP1000Ghg19-merge.bed +
HouseHGDP1000Ghg19-merge.bim + HouseHGDP1000Ghg19-merge.fam .
12890 variants loaded from .bim file.
8734 people (372

## Population stratification

Now, we will load the final dataset created before and run some populations stratification analyses (`PCA`, `MDS` and `ADMIXTURE`)

### PCA

In [16]:
os.chdir(os.path.join(outhouse, "05_ReferenceSamples"))
subprocess.run(["plink", "--bfile", "HouseHGDP1000Ghg19", "--pca", "50", "--pca-cluster-names", "0", "--within", "hgdp1000ghg19.fam", "--out", "PCA"])
for file in glob.glob("PCA.*"):
    shutil.move(file, os.path.join(projpath, "Results", "GenPCA", file))